In [1]:
from nltk.corpus import wordnet as wn

In [2]:
source = "F:/MSc DS/Project/glove.6B.50d.txt"

In [3]:
voc =[]
vector =[]
with open(source,encoding ='utf-8') as f:
    for line in f:
        linelist = line.split()
        voc.append(linelist[0])
        vec = linelist[1:]
        vec = [float(i) for i in vec]
        vector.append(vec)

In [4]:
# get the dic for all 400000tokens
n= len(voc)
dic = {}
for i in range(n):
    dic[voc[i]]=vector[i]

In [5]:
import pandas as pd
example = pd.read_csv('F:/MSc DS/Project/Github_resource/hypernymysuite/hypernymysuite-master/example.tsv',sep='\t',header =0)
m = example.shape[0]

In [6]:
example

,hypo,hyper,sim,is_oov
0,1840s,1850s,0.127720,0
1,1850s,1860s,0.193895,0
2,1880s,1890s,0.051568,0
3,1890s,1900s,0.065130,0
4,1910s,1920s,0.038549,0
...,...,...,...,...
41177,zoom,india,0.001010,0
41178,zoroastrianism,monotheism,0.021217,0
41179,zucchini,food,0.010976,0
41180,zucchini,novel,0.004437,0


In [7]:
a =list(example.hypo)
b = list(example.hyper)

In [8]:
c = a+ b
c =list(set(c))

In [9]:
len(c)

12254

In [10]:
example_voc = []
for i in c:
    if i in set(voc):
        example_voc.append(i)

In [11]:
total =len(example_voc)

In [12]:
total

12177

In [101]:
import re
import numpy as np
pattern = re.compile(r'\w+') 

def get_GaussianEmbedding(word):
    hypo = lambda s: s.hyponyms()
    syn =wn.synsets(word)
    if len(syn) ==0:
        mu = np.array(dic[word])[:,np.newaxis]
        sigma = mu*mu.transpose()+ np.identity(50)*0.00001
    else:
        w = wn.synsets(word)[0]
        l =list(w.closure(hypo))
        if len(l)<2:
            mu = np.array(dic[word])[:,np.newaxis]
            sigma = mu*mu.transpose()+ np.identity(50)*0.00001
        else:
            new_list =[]
            for i in l:
                if '_' in str(i):
                    continue
                m=pattern.match(str(i)[8:])
                if m != None:
                    new_list.append(m.group(0))
            if len(new_list) < 2:
                mu = np.array(dic[word])[:,np.newaxis]
                sigma = mu*mu.transpose()+ np.identity(50)*0.00001
            else:
                new_list2 =[]
                for i in new_list:
                    if i in set(voc):
                        ele = np.array(dic[i])[:,np.newaxis]
                        new_list2.append(ele)
                n =len(new_list2)
                if n < 2:
                    mu = np.array(dic[word])[:,np.newaxis]
                    sigma = mu*mu.transpose()+ np.identity(50)*0.00001
                else:
                    mu_total =np.zeros(shape=(50,1))
                    for i in new_list2:
                        mu_total = mu_total+i
                    mu =mu_total/n

                    sig_total = np.zeros(shape=(50,50))
                    for i in range(n):
                        sig = (new_list2[i]-mu)*(new_list2[i]-mu).transpose()
                        sig_total= sig_total+sig
                    sigma = sig_total/n + np.identity(50)*0.00001
    return (mu,sigma)
    

In [201]:
Gaussian_dic ={}
for i in range(total):
    Gaussian_dic[example_voc[i]]= get_GaussianEmbedding(example_voc[i])

In [202]:
len(Gaussian_dic)

12177

In [203]:

import pickle
with open('./WordNet_Gauss.pickle', 'wb') as e:
    pickle.dump(Gaussian_dic, e, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
import pickle
file = open('./WordNet_Gauss.pickle', 'rb')
Gauss_dic =pickle.load(file)
file.close()

In [25]:
import numpy as np

In [24]:
for i in range(12177):
    t =np.trace(Gauss_dic[example_voc[i]][1])
    norm = Gauss_dic[example_voc[i]][1]/t
    Gauss_dic[example_voc[i]] = (Gauss_dic[example_voc[i]][0],norm)

In [26]:
np.trace( Gauss_dic[example_voc[3]][1])

1.0

In [27]:
def KL_div(mui,muj, sigmai,sigmaj):
    m = np.linalg.inv(sigmaj)
    trace_fac = np.trace(np.dot(m,sigmai))
    mu_diff = np.dot(np.dot((muj-mui).transpose(),m),muj-mui)
    
    det_fac = np.linalg.det(sigmai)/np.linalg.det(sigmaj)
        
        # boound def_fac
    if det_fac < 1.0e-8:
        det_fac = 1.0e-8
    elif det_fac > 1.0e8:
        det_fac = 1.0e8
        
    return 0.5*(trace_fac + mu_diff - 50 - np.log(det_fac))

In [30]:
def get_repr(word1, word2):
    a = Gauss_dic[word1]
    b = Gauss_dic[word2]
    KL  = KL_div(a[0],b[0],a[1],b[1])
    result = 1/(1+KL)
    return float(result)

In [31]:
m = example.shape[0]

for i in range(m):
    if example.hypo[i] in Gauss_dic.keys() and example.hyper[i] in Gauss_dic.keys():
        example.sim[i]= get_repr(example.hypo[i], example.hyper[i])
    else:
        example.is_oov[i] =1

test =np.array(example).tolist()
import csv
with open(r'F:/MSc DS/Project/Github_resource/hypernymysuite/hypernymysuite-master/example/ke_wn.tsv','w',newline='') as f:
    tsv_w= csv.writer(f, delimiter='\t')
    tsv_w.writerow(['hypo','hyper','sim','is_oov'])
    tsv_w.writerows(test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [32]:
example

,hypo,hyper,sim,is_oov
0,1840s,1850s,7.727173e-07,0
1,1850s,1860s,1.744545e-06,0
2,1880s,1890s,1.962705e-06,0
3,1890s,1900s,8.887129e-07,0
4,1910s,1920s,1.251807e-07,0
...,...,...,...,...
41177,zoom,india,2.629060e-08,0
41178,zoroastrianism,monotheism,3.012417e-07,0
41179,zucchini,food,2.857585e-03,0
41180,zucchini,novel,4.553075e-08,0


In [35]:
example.sim[100:]

100      1.240773e-07
101      8.455430e-08
102      5.993039e-03
103      7.398951e-08
104      6.833433e-08
             ...     
41177    2.629060e-08
41178    3.012417e-07
41179    2.857585e-03
41180    4.553075e-08
41181    3.414076e-06
Name: sim, Length: 41082, dtype: float64

In [71]:
nans =[]
for i in range(m):
    if np.isnan(example.sim[i]):
        nans.append(i)

In [72]:
nans


[13678, 20062, 25809, 28414, 30966, 30986, 33910, 38657]

In [102]:
import re
import numpy as np
pattern = re.compile(r'\w+') 

def get_GaussianEmbedding2(word):
    hypo = lambda s: s.hyponyms()
    syn =wn.synsets(word)
    if len(syn) ==0:
        mu = np.array(dic[word])[:,np.newaxis]
        sigma = mu*mu.transpose()+ np.identity(50)*0.0001
    else:
        w = wn.synsets(word)[0]
        l =list(w.closure(hypo))
        if len(l)<2:
            mu = np.array(dic[word])[:,np.newaxis]
            sigma = mu*mu.transpose()+ np.identity(50)*0.0001
        else:
            new_list =[]
            for i in l:
                if '_' in str(i):
                    continue
                m=pattern.match(str(i)[8:])
                if m != None:
                    new_list.append(m.group(0))
            if len(new_list) < 2:
                mu = np.array(dic[word])[:,np.newaxis]
                sigma = mu*mu.transpose()+ np.identity(50)*0.0001
            else:
                new_list2 =[]
                for i in new_list:
                    if i in set(voc):
                        ele = np.array(dic[i])[:,np.newaxis]
                        new_list2.append(ele)
                n =len(new_list2)
                if n < 2:
                    mu = np.array(dic[word])[:,np.newaxis]
                    sigma = mu*mu.transpose()+ np.identity(50)*0.0001
                else:
                    mu_total =np.zeros(shape=(50,1))
                    for i in new_list2:
                        mu_total = mu_total+i
                    mu =mu_total/n

                    sig_total = np.zeros(shape=(50,50))
                    for i in range(n):
                        sig = (new_list2[i]-mu)*(new_list2[i]-mu).transpose()
                        sig_total= sig_total+sig
                    sigma = sig_total/n + np.identity(50)*0.0001
    return (mu,sigma)

In [ ]:
for i in range(12177):
    t =np.trace(Gauss_dic[example_voc[i]][1])
    norm = Gauss_dic[example_voc[i]][1]/t
    Gauss_dic[example_voc[i]] = (Gauss_dic[example_voc[i]][0],norm)

In [109]:
for i in nans:
    word1=example.hypo[i]
    word2=example.hyper[i]
    a =get_GaussianEmbedding2(word1)
    b =get_GaussianEmbedding2(word2)
    a1_norm = a[1]/np.trace(a[1])
    b1_norm = b[1]/np.trace(b[1])
    KL  = KL_div(a[0],b[0],a1_norm,b1_norm)
    result = float(1/(1+KL))
    example.sim[i] = result

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [110]:
nans =[]
for i in range(m):
    if np.isnan(example.sim[i]):
        nans.append(i)

In [111]:
test =np.array(example).tolist()
import csv
with open(r'F:/MSc DS/Project/Github_resource/hypernymysuite/hypernymysuite-master/example/ke_wn.tsv','w',newline='') as f:
    tsv_w= csv.writer(f, delimiter='\t')
    tsv_w.writerow(['hypo','hyper','sim','is_oov'])
    tsv_w.writerows(test)